In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import transformers
from transformers import AutoModelForSequenceClassification
from transformers import AutoModel

In [2]:
train_data = pd.read_csv("/kaggle/input/emnlp-task1/train.csv",encoding='utf8')
train_data.head(20)

,text,label
0,যে দেশে সন্ত্রাসরা দেশ চালায়সে দেশে শান্তি কিভ...,1
1,এই বিচার শেষ বিচার নয়।আসল বিচার হবে আল্লাহর আদ...,0
2,আরব দেশগুলোকে বলব ভারতের সাথে সব ব্যবসা বাণিজ্...,2
3,দেশটা সুস্থ নাই,0
4,আপনার কথা দুঃখ জনক আগে বিডিও থাকলে কেন ধরা হলন...,0
5,সত্যি এটা মুসলিম করেছে জেনে হিন্দু সম্প্রদায়ের...,0
6,একজন বাবা কতোটা অসহায় হলে এই কথা বলতে পারে আল্...,0
7,আবুজাহেলের বংশধর আওয়ামী লীগ বা হাসিনা লীগ জংগী...,1
8,সরকারের সব লোক ভারতের দালাল মনে রাখিছ আল্লাহ ছ...,1
9,এরা দালাল,1


In [3]:
val_data = pd.read_csv("/kaggle/input/emnlp-task1/dev.csv",encoding='utf8')
val_data.head(20)

,text,label
0,পাডা পুতার মাঝখানে পরে সাধারণ ২ মানুষের জিবন শ...,0
1,করোনার চাপে অনেক কিছু বন্ধ ও অনেক বিধি নিষেধ ক...,0
2,সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে য...,0
3,যে লোকটা মারা গেছে তার কি হবে তার দায়ভার কে ন...,0
4,নিউ মার্কেট এবং গুলিস্থান মার্কেটের ব্যবসায়ীর...,1
5,হাইয়া আলাল জিহাদ হাইয়া আলাল জিহাদ,2
6,ছাত্রলিগের বিচার চাই,0
7,এতদিন পর ঢাকা কলেজের ছাত্ররা উচিত কাজ করেছে। ন...,1
8,ওই দুই দোকানদারকে থেকে বিতাড়িত করা। তাহলেই তো...,0
9,"কথাগুলি শুনতে খারাপ শুনালেও বলতে পারি, যতই মেধ...",0


In [4]:
test_data = pd.read_csv("/kaggle/input/emnlp-task1-test-data/test_task1.csv",encoding='utf8')
test_data.head(20)

,text,label
0,বাংলাদেশের হিন্দুরা নিজেদের জন্য আলাদা হিন্দু ...,0
1,মাইজদী - চৌমুহুনী - ফেনী মন্দিরে হামলা নিয়ে রি...,0
2,"দয়া করে পবিত্র কুরআনুল কারিম বলেন,,,,পবিত্র কথ...",0
3,বিবিসি হলো সত্য কে বিনষ্টকারী আর মিথ্যা কে গ্র...,1
4,বুধবার কি তোরা মারা গেছিলি বিবিসি বাংলা,1
5,"কোরআন পাওয়া গেছে, কোরআন পাওয়া গেছে,যেমন ভাবে ব...",1
6,কোরআন কে কেন অবমাননা করা হল? মুসলিমরা কি দোষ ক...,0
7,আমরা হিন্দু রা কুরআন পূজা করি না। এটা সম্পূর্ণ...,0
8,সন্চালককে বলতে চাই কথাটি হবে দেব দেবীর পায়ে কো...,0
9,কোরান অবমাননা না বলে পাওয়া গেছে বলেন কেন?,0


In [6]:
train_data.shape

(2700, 2)

In [7]:
val_data.shape

(1330, 2)

In [8]:
test_data.shape

(2016, 2)

In [9]:
train_data[train_data['label']==0].shape

(1389, 2)

In [10]:
train_data[train_data['label']==1].shape

(922, 2)

In [11]:
train_data[train_data['label']==2].shape

(389, 2)

In [12]:
val_data[val_data['label']==0].shape

(717, 2)

In [13]:
val_data[val_data['label']==1].shape

(417, 2)

In [14]:
val_data[val_data['label']==2].shape

(196, 2)

In [15]:
test_data[test_data['label']==0].shape

(1096, 2)

In [16]:
test_data[test_data['label']==1].shape

(719, 2)

In [17]:
test_data[test_data['label']==2].shape

(201, 2)